In [1]:
%pip install plotly
%pip install pandas
%pip install nbformat --upgrade
'''pip install numpy
pip install lifetimes
pip install --upgrade matplotlib
pip install seaborn
pip install scikit-learn'''

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


'pip install numpy\npip install lifetimes\npip install --upgrade matplotlib\npip install seaborn\npip install scikit-learn'

In [2]:

import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors


In [6]:
data = pd.read_csv('C:\\Users\\Admin\\OneDrive\\Desktop\\Jupyter\\Online Retail\\online_retail.csv')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [8]:
data.dropna(subset = ['CustomerID'], inplace = True)

In [9]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['TotalAmount'] = (data['Quantity'] * data['UnitPrice']).astype('Float64').round(2)

In [10]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [11]:
reference_date = pd.Timestamp(dt.now().date())
#reference_date

In [12]:
reference_date = data['InvoiceDate'].max() + timedelta(days = 1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [14]:
'''
Recency : ลูกค้าที่สั่งซื้อสินค้าล่าสุดวันนี้ จะได้ R Score > ลูกค้าที่สั่งซื้อสินค้าล่าสุดเมื่อ 3 ปีที่แล้ว
Frequency : ลูกค้าที่สั่งซื้อสินค้าบ่อยที่สุดในร้าน จะได้ F Score > ลูกค้าmujสั่งซื้อสินค้าน้อยครั้งที่สุดในร้าน
Monetary : ลูกค้าที่ช้จ่ายเยอะที่สุดในร้าน จะได้ M Score > ลูกค้าที่ใช้จ่ายน้อยที่สุดในร้าน
'''
#ทำRFM Analysis โดย group by Customer ID
rfm = data.groupby('CustomerID').agg({
    'InvoiceDate' : lambda x: (reference_date - x.max()).days,
    'InvoiceNo' : 'count',
    'TotalAmount' : 'sum'
})

In [15]:
#ทำตารางRFM
rfm.rename(columns = {'InvoiceDate': 'Recency', 'InvoiceNo' : 'Frequency', 
                      'TotalAmount' : 'Monetary'}, inplace = True)
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.0
12347.0,2,182,4310.0
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.4


In [16]:
#ระบุ Quantiles
quantiles = rfm.quantile(q = [0.25, 0.50, 0.75])

#จัดคะแนนตามRFM
# x คือ ค่าที่จะกลายเป็นscore ในRFM
# p คือ column name ในRFM
# d คือ data frame ที่เก็บquantiles
def RScore(x, p, d):
    #Recency
    if (p == 'Recency'):
        if (x <= d[p][0.25]):
            return 4
        elif (x <= d[p][0.50]):
            return 3
        elif (x <= d[p][0.75]):
            return 2
        else:
            return 1
    # Frequency, Monetery
    else:
        if (x <= d[p][0.25]):
            return 1
        elif (x <= d[p][0.50]):
            return 2
        elif (x <= d[p][0.75]):
            return 3
        else:
            return 4
        
        
rfm['R'] = rfm['Recency'].apply(RScore, args = ('Recency', quantiles, ))
rfm['F'] = rfm['Frequency'].apply(RScore, args = ('Frequency', quantiles, ))
rfm['M'] = rfm['Monetary'].apply(RScore, args = ('Monetary', quantiles, ))

In [17]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.0,1,1,1
12347.0,2,182,4310.0,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.4,1,1,2


In [18]:
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis = 1)

In [19]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.0,1,1,1,111,3
12347.0,2,182,4310.0,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.4,1,1,2,112,4


In [20]:
#ระบุSegment

segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']

def assign_segment(score):
    if (score < 5):
        return 'Low-Value'
    elif (score < 9):
        return 'Mid-Value'
    else:
        return 'High-Value'
        
        
rfm['RFM_Segment_Labels'] = rfm['RFM_Score'].apply(assign_segment)

In [21]:
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Labels
CustomerID,,,,,,,,,
12346.0,326,2,0.0,1,1,1,111,3,Low-Value
12347.0,2,182,4310.0,4,4,4,444,12,High-Value
12348.0,75,31,1797.24,2,2,4,224,8,Mid-Value
12349.0,19,73,1757.55,3,3,4,334,10,High-Value
12350.0,310,17,334.4,1,1,2,112,4,Low-Value


In [22]:
segment_counts = rfm['RFM_Segment_Labels'].value_counts().reset_index()
segment_counts.columns = ['RFM_Segment', 'Count']
segment_counts = segment_counts.sort_values('RFM_Segment')

In [23]:
#Create Bar chart using plotly

fig = px.bar(segment_counts,
             x = 'RFM_Segment', 
             y = 'Count', 
             title = "Customer Distribution by RFM Segment",
             labels = {'RFM_Segment' : 'RFM_Segment', 'Count' : 'Number of Customers'},
             color = 'RFM_Segment', 
             color_discrete_sequence = px.colors.qualitative.Pastel)


fig.show()

In [55]:
#Create RFM chart
rfm['RFM_Customer_Segments'] = ''

rfm.loc[(rfm['RFM_Score'] >= 9), 'RFM_Customer_Segments'] = "VIP/Loyal"
rfm.loc[(rfm['RFM_Score'] >= 6) & (rfm['RFM_Score'] < 9), 'RFM_Customer_Segments'] = "Potential Loyalist"
rfm.loc[(rfm['RFM_Score'] >= 5) & (rfm['RFM_Score'] < 6), 'RFM_Customer_Segments'] = "At Risk"
rfm.loc[(rfm['RFM_Score'] >= 4) & (rfm['RFM_Score'] < 5), 'RFM_Customer_Segments'] = "Can't Lose"
rfm.loc[(rfm['RFM_Score'] >= 3) & (rfm['RFM_Score'] < 4), 'RFM_Customer_Segments'] = "Lost"

segment_counts = rfm['RFM_Customer_Segments'].value_counts().sort_index()

In [56]:
segment_product_counts = rfm.groupby(['RFM_Segment_Labels', 
                                      'RFM_Customer_Segments']).size().reset_index(name = 'Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending = False)

In [57]:
#Create Treemap

fig_treemap_segment_product = px.treemap(segment_product_counts, 
                                         path = ['RFM_Segment_Labels', 'RFM_Customer_Segments'], 
                                         values= 'Count', color = 'RFM_Segment_Labels',
                                         color_discrete_sequence = px.colors.qualitative.Pastel,
                                         title = "RFM Customer Segment by Monetary")

#Show Treemap
fig_treemap_segment_product.show()

In [58]:
vip_segment = rfm[rfm['RFM_Customer_Segments'] == "VIP/Loyal"]

In [59]:
fig = go.Figure()
fig.add_trace(go.Box(y = vip_segment['Recency'], name = "Recency"))
fig.add_trace(go.Box(y = vip_segment['Frequency'], name = "Frequency"))
fig.add_trace(go.Box(y = vip_segment['Monetary'], name = "Monetary")) 


In [60]:
correlation_matrix = vip_segment[['R', 'F', 'M']].corr()

In [62]:
#Create Heatmap

fig_heatmap = go.Figure(data = go.Heatmap(
                    z = correlation_matrix.values, 
                    x = correlation_matrix.columns, 
                    y = correlation_matrix.columns,
                    colorscale = 'RdBu',
                    colorbar = dict(title = 'Correlation')))

fig_heatmap.update_layout(title = "Correlation Matrix of RFM Monetary within Champion Segment")


#Show Heatmap

fig_heatmap.show()

In [64]:
# ดูall of rank
pastel_colors = plotly.colors.qualitative.Pastel

fig = go.Figure(data = [go.Bar(x = segment_counts.index, y = segment_counts.values,
                               marker = dict(color = pastel_colors))])


vip_color = 'rgb(158, 202, 225)'

fig.update_traces(marker_color=[vip_color if segment == "Champions" else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color = 'rgb(8, 48, 107)',
                  marker_line_width = 1.5, opacity = 0.6)


# update the layout
fig.update_layout(title = "Comparison of RFM Segments",
                  xaxis_title = "RFM Segments",
                  yaxis_title = "Number of Customers",
                  showlegend = False)

# Display the figure

fig.show()

In [69]:
segment_scores = rfm.groupby('RFM_Customer_Segments')[['R', 'F', 'M']].mean().reset_index()
fig = go.Figure()


#Add bar for Recency Score
fig.add_trace(go.Bar(
    x = segment_scores['RFM_Customer_Segments'],
    y = segment_scores['R'],
    name = 'Recency Score',
    marker_color = 'rgb(158, 202, 225)'
))

#Add bar for Frequency Score
fig.add_trace(go.Bar(
    x = segment_scores['RFM_Customer_Segments'],
    y = segment_scores['F'],
    name = 'Frequency Score',
    marker_color = 'rgb(94, 158, 217)'
))

#Add bar for Monetary Score
fig.add_trace(go.Bar(
    x = segment_scores['RFM_Customer_Segments'],
    y = segment_scores['M'],
    name = 'Monetary Score',
    marker_color = 'rgb(32, 102, 148)'
))


# update the layout
fig.update_layout(title = "Comparison of RFM Segments based on Recency, Frequency, Monetery Scores",
                  xaxis_title = "RFM Segments",
                  yaxis_title = "Score",
                  barmode = "group",
                  showlegend = True)

# Display the figure

fig.show()

In [71]:
data.to_csv("Cleaned_Data.csv", index = False)